In [3]:
import apache_beam as beam

p = beam.Pipeline()

side_list = []
with open('source datasets/customers_exclude.txt', 'r') as exclude_file:
    for cust_id in exclude_file:
        side_list.append(cust_id.rstrip())

print(side_list)


class SplitRow(beam.DoFn):
    def process(self, element, side_list):
        customer = element.split(',')
        if customer[0] not in side_list:
            return [customer]


class ProcessCustomers(beam.DoFn):
  def process(self,element,country,start_char):
    if(element[2]=='NY'):
      yield  element
    else:
      yield  beam.pvalue.TaggedOutput('Other_Cust',element)
    if(element[1].startswith('J')):
       yield  beam.pvalue.TaggedOutput('Names_J',element)
  


customers = (
    p
    |beam.io.ReadFromText('source datasets/Customers_age.txt')
    |beam.ParDo(SplitRow(),side_list)
    |beam.ParDo(ProcessCustomers(),country='NY',start_char='J').with_outputs('Names_J','Other_Cust',main='NewYork_Cust')
)

newyork_customers = customers.NewYork_Cust
other_customers = customers.Other_Cust
customer_withname_J = customers.Names_J

newyork_customers | 'Write Newyork Customers PCollection' >> beam.io.WriteToText("output/10/newyork")
other_customers  | 'Write Customers PCollection that lives in other cities' >> beam.io.WriteToText("output/10/customers_other_cities")
customer_withname_J  | 'Write Customers names with J PCollection' >> beam.io.WriteToText("output/10/customers_names_j")



p.run()

['3', '7', '10']
